In [2]:
from caffe2.python import workspace,brew,model_helper
import numpy as np

In [6]:
# make random data
data = np.random.rand(16,100).astype(np.float32)
label = (np.random.rand(16)*10).astype(np.int32)
workspace.FeedBlob('data', data)
workspace.FeedBlob('label', label)

# Use brew module to build a MLP
model = model_helper.ModelHelper('model_mlp')
fc = brew.fc(model, 'data', 'fc', 100, 50)
relu = brew.relu(model, fc, 'relu')
pred = brew.fc(model, relu, 'pred', 50, 10)
softmax = brew.softmax(model, pred, 'softmax')
# Add accuracy and loss  
accuracy = model.Accuracy([softmax, 'label'], 'accuracy')
xentropy = model.LabelCrossEntropy([softmax, 'label'], 'x_entropy')
model.AddGradientOperators([xentropy])
print model.net.Proto()
print model.param_init_net.Proto()

name: "model_mlp"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc"
  name: ""
  type: "FC"
  arg {
    name: "use_cudnn"
    i: 1
  }
  arg {
    name: "order"
    s: "NCHW"
  }
  arg {
    name: "cudnn_exhaustive_search"
    i: 0
  }
}
op {
  input: "fc"
  output: "relu"
  name: ""
  type: "Relu"
  arg {
    name: "cudnn_exhaustive_search"
    i: 0
  }
  arg {
    name: "order"
    s: "NCHW"
  }
  engine: "CUDNN"
}
op {
  input: "relu"
  input: "pred_w"
  input: "pred_b"
  output: "pred"
  name: ""
  type: "FC"
  arg {
    name: "use_cudnn"
    i: 1
  }
  arg {
    name: "order"
    s: "NCHW"
  }
  arg {
    name: "cudnn_exhaustive_search"
    i: 0
  }
}
op {
  input: "pred"
  output: "softmax"
  name: ""
  type: "Softmax"
  arg {
    name: "order"
    s: "NCHW"
  }
  arg {
    name: "cudnn_exhaustive_search"
    i: 0
  }
  engine: "CUDNN"
}
op {
  input: "softmax"
  input: "label"
  output: "accuracy"
  name: ""
  type: "Accuracy"
}
op {
  input: "softmax"
  input:

In [7]:
# Run net
workspace.RunNetOnce(model.param_init_net)
workspace.CreateNet(model.net, overwrite=True)
for _ in range(100):
    # make random data
    data = np.random.rand(16,100).astype(np.float32)
    label = (np.random.rand(16)*10).astype(np.int32)
    workspace.FeedBlob('data', data)
    workspace.FeedBlob('label', label)
    
    workspace.RunNet(model.net, 10)
    print workspace.FetchBlob('accuracy')
    print workspace.FetchBlob('x_entropy')

0.125
[2.4332201 2.7447758 2.6628475 2.7024913 2.7417455 2.2447755 1.6154557
 2.6210237 3.2376375 1.605381  1.5898992 1.9574531 2.6158197 2.0713782
 1.0213393 2.3993964]
0.1875
[2.6915724 2.736333  3.031958  2.5659175 1.7201682 3.0332298 1.490619
 2.9481168 1.9232118 2.719979  2.9890373 1.5825956 1.5000614 1.9342363
 1.6900427 3.205124 ]
0.0
[2.552296  2.4681764 2.5394902 2.8803499 3.6367528 1.5592557 2.4578595
 2.9405973 2.7634943 2.936967  2.2945983 2.5099173 1.9002016 2.8684585
 2.5723865 2.5791786]
0.0625
[2.6154244 2.8631344 1.6536565 1.5686783 2.83204   2.8809159 1.7353271
 1.68864   3.11597   2.780994  1.7840282 2.2716777 2.6796467 2.887498
 3.2506716 2.0145414]
0.125
[2.74493   2.9005663 1.6691173 3.45122   2.2283607 2.9597273 2.4275553
 2.6492004 3.0276208 3.3122222 1.7669168 1.381348  2.6824148 2.557319
 2.857892  2.8773775]
0.25
[1.9652281 2.886063  2.177266  2.3189037 2.5316043 1.9415245 1.3946891
 2.8272405 2.1675158 1.6384453 3.112617  1.4840796 3.1388767 3.0811808
 2.079